transformers를 사용하여 pretransed된 bert모델 생성

In [2]:
import torch
import os
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer
import json

In [4]:
import json

# 비속어 사전 01
slang_list_01 = []
f = open("./List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words/ko", 'r')
lines = f.readlines()
for line in lines:
    line = line.replace("\n", "")
    slang_list_01.append(line)
f.close()

# 비속어 사전 02
slang_list_02 = []
f = open("./korean-bad-words/korean-bad-words.md", 'r')
lines = f.readlines()
for line in lines:
    line = line.replace("\n", "")
    slang_list_02.append(line)
f.close()

# 비속어 사전 03
with open('./Gentleman/resources/badwords.json') as json_file:
    json_data = json.load(json_file)
slang_list_03 = json_data["badwords"]

# 비속어 사전 통합
slang_list = slang_list_01 + slang_list_02 + slang_list_03
slang_list = list(set(slang_list))

In [5]:
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [7]:

   
import torch
import torch.nn as nn

input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [8]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)

NameError: name 'input_size' is not defined

Data 불러오기

In [47]:
Datas = pd.read_csv('/opt/ml/jh/ChatbotData.csv')

In [58]:
chatbot_Q = Datas['Q']
chatbot_A = Datas['A']

In [60]:
chatbot_Q_list = []
for data in Datas['Q']:
  chatbot_Q_list.append(data)

chatbot_A_list = []
for data in Datas['A']:
  chatbot_A_list.append(data)

chatbot_Q_list.append('비속어가 포함된 문장입니다.')
chatbot_A_list.append('위로봇에 의해 삭제된 문장입니다.')

In [64]:
#print(chatbot_A_list[:])

Data전처리

In [65]:
def get_cls_token(sent) : 
  model.eval()
  tokenized_sent = tokenizer(
      sent,
      return_tensors = "pt",
      truncation = True,
      add_special_tokens=True,
      max_length = 128
  )
  with torch.no_grad():
    outputs = model(**tokenized_sent)
  logits = outputs.last_hidden_state[:,0,:].detach().cpu().numpy()
  return logits

In [80]:
def find_Badword(sent) :
    for word in slang_list :
        if word in sent :
            sent = '비속어가 포함된 문장입니다.'
    return sent

코사인 유사도를 활용하기 위해 import

In [66]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

dataset_cls_hidden = []

#질문만 hidden_state계산
for q in tqdm(chatbot_Q_list) :
  q_cls = get_cls_token(q)
  dataset_cls_hidden.append(q_cls)
  # print(dataset_cls_hidden)
dataset_cls_hidden = np.array(dataset_cls_hidden).squeeze(axis=1)

100%|██████████| 11824/11824 [04:47<00:00, 41.09it/s]


In [53]:
#코사인 유사도를 활용하여 정확도 계산

In [87]:
question = '존나 뭐야'
question = find_Badword(question)
question_cls_hidden = get_cls_token(question)

In [90]:
cos_sim = cosine_similarity(question_cls_hidden, dataset_cls_hidden)

In [89]:
top_question = np.argmax(cos_sim)
print('나 : ',question)
print('챗봇: ', chatbot_A_list[top_question])

나 :  비속어가 포함된 문장입니다.
챗봇:  위로봇에 의해 삭제된 문장입니다.
